### Install

In [268]:
#pip install ucimlrepo

In [269]:
#pip install psycopg2

### Imports

In [288]:
# Imports
from ucimlrepo import fetch_ucirepo
import pandas as pd
import psycopg2
from collections import defaultdict
from scipy.stats import entropy
from scipy import stats as kl
from decimal import Decimal
from scipy.stats import entropy
import numpy as np


In [271]:
connection = psycopg2.connect(database="Project645", user="vaishnavishah", password="10jan2001", host="localhost", port=5432)

cursor = connection.cursor()


In [272]:
# cursor.execute("SELECT * from unmarried;")
# record = cursor.fetchall()
# unmarried_df = pd.DataFrame(record, columns=['age', 'workclass', 'fnlwgt', 'education', 'education-num',
#        'occupation', 'relationship', 'race', 'sex',
#        'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'])

In [273]:
# unmarried_df

In [274]:
# query = "select workclass, sum(age) from married where workclass is not null group by workclass order by workclass;"

# cursor.execute(query)
# record = cursor.fetchall()
# print(record)

In [275]:
# cursor.execute("SELECT * from married;")
# record = cursor.fetchall()
# married_df = pd.DataFrame(record, columns=col_names)

In [276]:
results_kl = defaultdict(list)

In [277]:
views = []
continuous=['age','fnlwgt', 'education_num','capital_gain','capital_loss','hours_per_week']
discrete=['workclass','education','occupation','relationship','race','sex','native_country']
aggregate=['sum','avg','max','min','count']

k=10
num_partitions=20
delta= 0.01

In [290]:
def aggregate_values(o_1, o_2):
    dict_res = defaultdict(list)
    f_list_1 = []
    f_list_2 = []

    # Handle input 1
    if isinstance(o_1, dict):
        items_1 = o_1.items()
    else:  # cursor
        items_1 = o_1.fetchall()

    # Handle input 2
    if isinstance(o_2, dict):
        items_2 = o_2.items()
    else:  # cursor
        items_2 = o_2.fetchall()

    for k, v in items_1:
        if v != 0:
            dict_res[k].append(v)
        else:
            dict_res[k].append(Decimal(1e-10))

    for k, v in items_2:
        if k not in dict_res:
            dict_res[k].append(Decimal(1e-10))
        if v != 0:
            dict_res[k].append(v)
        else:
            dict_res[k].append(Decimal(1e-10))

    for k, v in dict_res.items():
        if len(v) != 2:
            dict_res[k].append(Decimal(1e-10))

    for i in dict_res.values():
        f_list_1.append(i[0])
        f_list_2.append(i[1])

    return f_list_1, f_list_2

def get_utility(f_list_1, f_list_2):
    # Convert Decimal objects to float
    f_list_1 = [float(x) for x in f_list_1]
    f_list_2 = [float(x) for x in f_list_2]

    # Compute normalized lists
    sum_1 = sum(f_list_1)
    sum_2 = sum(f_list_2)
    norm_1 = np.array(f_list_1) / sum_1
    norm_2 = np.array(f_list_2) / sum_2

    # Calculate entropy
    utility = entropy(norm_1, norm_2)

    return utility

In [293]:
all_views = []
for a in aggregate:
    for d in discrete:
        for c in continuous:
            all_views.append((a,d,c))

kl_res=defaultdict(list)
for a,d,c in all_views:
    query1 = 'select '+d+', '+a+'('+c+') from married where '+d+' is not null group by '+d+' order by '+d+';'
    query2 = 'select '+d+', '+a+'('+c+') from unmarried where '+d+' is not null group by '+d+' order by '+d+';'
    cursor.execute(query1)
    query1_res = cursor
    cursor.execute(query2)
    query2_res = cursor
    res1, res2 = aggregate_values(query1_res, query2_res)
    uti = get_utility(res1, res2)
    kl_res[a,d,c].append(uti)

top_k = sorted(kl_res.items(), key= lambda item: item[1], reverse=True)[:k]

In [294]:
top_k

[(('min', 'native_country', 'capital_loss'), [3.7135720666467606]),
 (('sum', 'native_country', 'capital_gain'), [3.3605794779297864]),
 (('sum', 'native_country', 'capital_loss'), [3.3300353157652993]),
 (('sum', 'native_country', 'education_num'), [3.2429473940288993]),
 (('sum', 'native_country', 'age'), [3.2137577579000687]),
 (('count', 'native_country', 'age'), [3.206708854007431]),
 (('count', 'native_country', 'fnlwgt'), [3.206708854007431]),
 (('count', 'native_country', 'education_num'), [3.206708854007431]),
 (('count', 'native_country', 'capital_gain'), [3.206708854007431]),
 (('count', 'native_country', 'capital_loss'), [3.206708854007431])]